In [7]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"
openai.api_key = 'XXXXXXXXXXXXXXXXXX'

### Reading open source data - from openai

In [16]:
df = pd.read_csv('https://cdn.openai.com/API/examples/data/olympics_sections_text.csv')
df['content']=df['title']+' '+df['heading']+ ' '+df['content']
df = df.set_index(["title", "heading"])
print(f"{len(df)} rows in the data.")
df.sample(5)

3964 rows in the data.


,,content,tokens
title,heading,,
Argentina at the 2020 Summer Olympics,Artistic,Argentina at the 2020 Summer Olympics Artistic...,129
2020 CONCACAF Women's Olympic Qualifying Championship qualification,Caribbean,2020 CONCACAF Women's Olympic Qualifying Champ...,47
Field hockey at the 2020 Summer Olympics – Men's qualification,Olympic qualifying events,Field hockey at the 2020 Summer Olympics – Men...,86
2020 CAF Women's Olympic Qualifying Tournament,Third round,2020 CAF Women's Olympic Qualifying Tournament...,51
Fencing at the 2020 Summer Olympics – Women's team sabre,Background,Fencing at the 2020 Summer Olympics – Women's ...,177


### Getting embeddings

In [9]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

In [17]:
document_embeddings = compute_doc_embeddings(df.head(100))

### We could directly use the dataframe for QA, but because of the token limitations for the LLMs, we have to first get a most relevant context based on the question from the given dataframe,and then pass that with question in the formatted prompt to get the desired answer.

In [10]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [11]:
#Configs
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

### Formatted prompt creation

In [12]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

### Getting desired answer 

In [13]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [20]:
answer_query_with_context("Why was the 2020 Summer Olympics originally postponed?", df, document_embeddings)

Selected 1 document sections:
('2020 Summer Olympics', 'Impact of the COVID-19 pandemic')


'The 2020 Summer Olympics was originally postponed due to concerns about the potential impact of the COVID-19 pandemic on athletes and visitors to the Olympic Games.'